https://pythonspeed.com/
Классный ресурс по оптимизации вычислений и работе с памятью

# Ускорение вычислений
Для того, чтобы проверять скорость вычислений в jupyter можно использовать либо %%time перед всем кодом в ячейке,
либо %time или %timeit перед строчкой выполнения кода

In [6]:
%%time
x = sum((i for i in range(100000)))

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 5.37 ms


*Wall time* показывает суммарное время работы вашего кода.
*CPU times* показываем суммарное время, когда ваш процессор был занят.
При этом *sys* - это время, связанное с обращениями к операционной системе.

Если *wall time* превышает *CPU time*, то это значит, что программа ждет еще каких-то ответов, помимо вычислений процессора.

Таким образом, возможны следующие соотношения:

1. CPU time/ wall time  ≈ 1: Процесс тратит все время на использование CPU. Более быстрый CPU позволит ускорить данный процесс.
2. CPU time/ wall time < 1: Чем ниже отношение, тем больше CPU тратит на ожидание ответа (сети, жесткого диска и т.д., в том числе и просто режим сна).
Например, если он равен 0.75, то 25% времени CPU тратит на ожидание.

In [7]:
%time
sum([i for i in range(10000)])
%time
l = [a for a in range(10000)]

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.58 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [8]:
%timeit sum((i for i in range(10000)))
%timeit sum([i for i in range(10000)])

450 µs ± 6.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
274 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Numpy

Данный пакет написан на С++ и позволяет совершать самые быстрые численные операции на всем диком Python.

In [2]:
import numpy as np
%timeit np.sum(np.array((i for i in range(10000))))
%timeit np.sum(np.array([i for i in range(10000)]))
%timeit sum((i for i in range(10000)))

5.97 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
751 µs ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
452 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%timeit np.sum(np.fromiter((i for i in range(10)), dtype=int))
%timeit np.sum([i for i in range(10)])
%timeit sum((i for i in range(10)))

5 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.74 µs ± 50.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
786 ns ± 7.35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
# Индексация
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(a)
print(a[0, 1])
print(a[:2, :3])
print(a[1:2, 2:3])
print(a[1:, 2:])
print(a[1, :])
print(a[1:2, :])
print(a[[1], :])
print(a[:, 1])
print(a[:, 1:2])

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
2
[[1 2 3]
 [5 6 7]]
[[7]]
[[ 7  8]
 [11 12]]
[5 6 7 8]
[[5 6 7 8]]
[[5 6 7 8]]
[ 2  6 10]
[[ 2]
 [ 6]
 [10]]


In [28]:
# Размерность массивов
b = np.array([[1,2,3], [4,5,6], [7,8,9]])
print(b.shape)

# Изменение размерности (умноженные размерности совпадают)
v = np.array([1,2,3])
print(v, "\n", v.reshape(3, 1))

print(np.arange(16).reshape(2, 4, 2))
print(np.arange(16).reshape(1, 4, 2, 2))

# Изменение размерности (новая размерность другая - заполнение нулями)
a0 = np.arange(4)
a0.resize((8,))
print(a0)

(3, 3)
[1 2 3] 
 [[1]
 [2]
 [3]]
[[[ 0  1]
  [ 2  3]
  [ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]
  [12 13]
  [14 15]]]
[[[[ 0  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]

  [[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]]
[0 1 2 3 0 0 0 0]


In [12]:
# Сортировка

c = np.array([[4, 3, 5], [1, 2, 1]])
print(np.sort(c, axis=1))

# Возвращает индексы элементов, которые были отсортированы
j = np.argsort(c)
print(j)
print(c[:, j])

[[3 4 5]
 [1 1 2]]
[[1 0 2]
 [0 2 1]]
[[[3 4 5]
  [4 5 3]]

 [[2 1 1]
  [1 1 2]]]


In [13]:
# Присвоение по условию (если, то)
data = np.random.randn(7, 4)
np.where(data, data>0, 0)

array([[1, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 0],
       [1, 1, 0, 0]])

In [34]:
# Слияние массивов
x,y,z = np.arange(1,3), np.arange(3,5), np.arange(5,7)
print(np.concatenate([x,y,z]))

array2D_1 = np.array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])
array2D_2 = np.array([[10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]])

# Вертикальное слияние
print(f"Вертикальное : \n {np.vstack((array2D_1, array2D_2))}")

# Горизонтальное слияние
print(f"Горизонтальное : \n {np.hstack((array2D_1, array2D_2))}")

[1 2 3 4 5 6]
Вертикальное : 
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [10 11 12]
 [13 14 15]
 [16 17 18]]
Горизонтальное : 
 [[ 0  1  2 10 11 12]
 [ 3  4  5 13 14 15]
 [ 6  7  8 16 17 18]]


In [38]:
# Применение функции к разным осям массива
m = np.random.randn(3,2)
# Сначала указываем функцию, потом по какой оси, а затем указываем сам массив
print(np.apply_along_axis(lambda x: sum(x ** 2), 1, m))
print(np.apply_along_axis(lambda x: sum(x ** 2), 0, m))
# Обратим внимание, что у функции может быть только один аргумент - это значения массива из конкретной оси

[0.31300339 0.07865637 0.67738013]
[0.58474968 0.4842902 ]


In [41]:
# Траблы с округлением при сравнении
print(np.round(0.3, 15) == np.round(3 * 0.1, 15))
print(np.round(0.3, 16) == np.round(3 * 0.1, 16))
print(np.round(0.3, 17) == np.round(3 * 0.1, 17))
# Используем такую функцию, чтобы проверить, равны ли массивы
print(np.allclose(0.3, 3*0.1))

True
True
False
True


# Работа с памятью

Помним, что у любого компьютера есть ограничение по оперативной памяти, а все наши рабочие процессы хранятся именно там.
Любой сайт - это такой же компьютер, как и у вас, со своим жестким диском и оперативной памятью.
Таким образом, помимо скорости вычислений, нам бы хотелось еще и не вылазить за пределы оперативной памяти.
В противном случае, программа просто прекратит свою работу и придется запускать все заново.


In [6]:
import sys
data = np.random.randn(7, 4, 10000)
sys.getsizeof(data)  # Данная команда показывает, сколько байтов содержит датасет

2240136

In [4]:
sys.getsizeof(data) / 2 ** 20

2.1363601684570312

## Garbage collector

https://stackify.com/python-garbage-collection/

В Python, в отличие от более ранних языков программирования, не нужно самим напрямую управлять памятью.
Здесь реализовано автоматическое управление памятью.
Очевидный плюс для разработчика - можно быстрей писать программы, не тратя время на выделение памяти для каждой переменной.
Очевидный минус - если мы совсем перестаем контроллировать этот процесс, то может выйти за пределы оперативной памяти.

Существует два основных аспекта garbage collection в CPython:

- Reference counting
- Generational garbage collection

### Reference counting garbage collection

Где бы не создавался питоновский объект, базовый С объект использует питоновский тип (например, лист, словарь или функцию) и количество отсылок.
Чем чаще мы отсылаемся к какому-то объекту, тем больше накапливается этого показателя.
Основные способы сослаться на какой-либо объект:
- присвоить объект переменной
- добавить объект к структуре данных (например, добавление в список или как *property* в классе)
- вставить объект как аргумент функции

In [23]:
c1 = "new_var"
sys.getrefcount(c1)

2

In [14]:
a = 'my-string'
sys.getrefcount(a)
# Всего было 3 отсылки к объекту a - создание раннее, переприсваивание сейчас и вызов в функции sys.getrefcount

3

In [24]:
b = [a] # Make a list with a as an element.
c = { 'key': a } # Create a dictionary with a as one of the values.
sys.getrefcount(a)

4

### Generational garbage collection



In [25]:
class MyClass(object):
       pass
a = MyClass()
a.obj = a
del a
a

NameError: name 'a' is not defined

Здесь мы определили новый класс.
Затем создали экземпляр класса и присвоили экземпляру аттрибут в качестве самого класса. В итоге мы удалили объект.
Несмотря на то, что объект мы уже удалили, в Python все еще хранится экземпляр в памяти.
Такую проблему мы называем исходный цикл (reference cycle). Эта и многие другие проблемы решаются при помощи модуля *gc*.

Существуют два основных понятия для garbage collection:
- поколение (generation)
- порог (threshold)

Garbage collector отслеживает абсолютно все объекты, которые когда-либо попадали в Python.
Всего существует 3 поколения для объектов и объекты в зависимости от наших действий передвигаются от старого к более новому.
Для каждого поколения *gc* модуль выделены пороги для числа объектов. Если число объектов переваливает за этот порог,
то *gc* запустит процесс сбора. Любые объекты, которые переживут этот процесс, попадут в более старое поколение.

В отличие от проблемы с отсылками, здесь мы можем управлять этим процессом.
Во-первых, мы можем изменять пороги, которые влияют на вызов процесса сборки мусора.
Во-вторых, можем вручную вызывать сборщик мусора или, наоборот, отключать его.

In [26]:
import gc
print(gc.get_threshold())
print(gc.get_count())

(700, 10, 10)
(134, 5, 2)


In [27]:
gc.collect()

10

In [28]:
gc.get_count()

(246, 0, 0)

In [38]:
# Изменим порог для перехода в другие поколения
gc.set_threshold(1000, 15, 15)
gc.get_threshold()

(1000, 15, 15)

Увеличение порога сократит частоту запуска сборщика мусора.
Это ускорит наши процессы, но все объекты будут дольше храниться в памяти.

## Типы данных

Вспомним, что Python - динамический язык программирования. Однако, для более ручного управления памятью и кодом, мы можем объектам присваивать
типы данных самостоятельно. Рекомендуется использовать типы данных *numpy* массивов.

1. int - целочисленный тип данных: (-2, -1, 0, 1, ....)
2. uint - целочисленный тип данных: (0, 1, 2, .....)
3. float - вещественный тип данных с плавающей точкой
4. bool - логический тип данных
5. string - строковый тип данных

После int, uint, float мы можем указать числа, которые будут отвечать за длину диапазона возможных значений.
Например, np.int32 означает, что диапазон значений включает в себя 2 ** 32 (около 4 млрд) чисел, которые расположены симметрично относительно нуля;
np.uint8 - диапазон чисел от 0 до 255

Отметим, что в случае np.float32, np.float64 активируются Ctype типы данных. Это значит, что если нам нужна скорость обработки, то
мы должны использовать именно эти типа данных для вещественных чисел. В противном случае, можно использовать float16


1000

In [37]:
print(np.uint8(255))
print(np.uint8(1000))  # 256 + 256 + 256 + 232
print(np.float64(5/3))
print(np.float16(5/3))

255
232
1.6666666666666667
1.667


Давайте позапускаем разные коды и посмотрим, сколько уходит памяти в каждой строчке.

In [3]:
import resource

usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss  # выдает значения в Кб
print(usage)

151908352


In [4]:
from memory_checking_1 import make_big_array, make_two_arrays
ar = make_big_array()
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

151949312


In [50]:
# Peak memory usage
ar1, ar2 = make_two_arrays()
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 2 ** 10)

177.78125


In [51]:
print(sys.getsizeof(ar1) / 2 ** 20)
print(sys.getsizeof(ar2) / 2 ** 20)

80.00012969970703
80.00012969970703


In [52]:
import tracemalloc
tracemalloc.start()
make_two_arrays()
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / (1024 * 1024)}MB; Peak was {peak / (1024 * 1024)}MB")
tracemalloc.stop()

Current memory usage is 0.0008916854858398438MB; Peak was 160.00118160247803MB


In [53]:
def memory_analyze(func, result="Mb", *args):
    tracemalloc.start()
    func(*args)
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    if result == "Mb":
        print(f"Current memory usage is {current / (1024 * 1024)}MB; Peak was {peak / (1024 * 1024)}MB")
    elif result == "Gb":
         print(f"Current memory usage is {current / (1024 * 1024 * 1024)}Gb; Peak was {peak / (1024 * 1024 * 1024)}Gb")
    elif result == "Kb":
        print(f"Current memory usage is {current / 1024}KB; Peak was {peak / 1024}KB")
    else:
        ValueError("Insert correct type size")

    return
memory_analyze(make_big_array, result="Gb")

Current memory usage is 4.231929779052734e-06Gb; Peak was 0.3906293138861656Gb


## Учимся экономить память

### Проблемы с локальными переменными в функции

In [53]:
def load_1GB_of_data():
    return np.ones((2 ** 30), dtype=np.uint8)

def process_data():
    data = load_1GB_of_data()
    return modify2(modify1(data))

def modify1(data):
    return data * 2

def modify2(data):
    return data + 10

memory_analyze(process_data, "Gb")

Current memory usage is 1.5990808606147766e-06Gb; Peak was 3.000001952983439Gb


Как так вышло? Мы создали данные объемом 1 Гб. Два раза умножили на 2 и прибавили и объем наших данных увеличился в 3 раза.
По-хорошему, объем должен был вырасти до 2 Гб. Однако, откуда здесь еще лишний 1 Гб?
Весь прикол как раз в сборщике мусора, который учел количество обращений в объекту не 2, а 3 раза!
Есть 3 способа борьбы с этим

In [54]:
# 1. Не использовать локальную переменную
def process_data():
    return modify2(modify1(load_1GB_of_data()))
memory_analyze(process_data, "Gb")

Current memory usage is 1.1557713150978088e-06Gb; Peak was 2.000001427717507Gb


In [55]:
# 2. Переприсвоить локальной переменной
def process_data():
    data = load_1GB_of_data()
    data = modify1(data)
    data = modify2(data)
    return data
memory_analyze(process_data, "Gb")

Current memory usage is 1.239590346813202e-06Gb; Peak was 2.0000014854595065Gb


In [ ]:
# 3. Передача права собственности на объект
class Owner:
    def __init__(self, data):
        self.data = data

def process_data():
    data = Owner(load_1GB_of_data())
    return modify2(modify1(data))

def modify1(owned_data):
    data = owned_data.data
    # Remove a reference to original data:
    owned_data.data = None
    return data * 2




### Излишняя модификация и присваивание

In [ ]:
np.random.randint(1, 10, 10000)

In [68]:
#
def normalize(array: np.ndarray) -> np.ndarray:
    """
    Takes a floating point array.

    Returns a normalized array with values between 0 and 1.
    """
    low = array.min()
    high = array.max()
    return (array - low) * (high - low)
memory_analyze(normalize, "Kb", np.random.randint(1, 10, 10000).reshape(10, 100, 10))

Current memory usage is 117.0244140625KB; Peak was 273.6025390625KB


In [71]:
# In-place type of assigning
def normalize_in_place(array: np.ndarray):
    low = array.min()
    high = array.max()
    result = array.copy()  # Это нужно для того, чтобы не менять значения исходного массива!
    array -= low
    array *= high - low
    return array
memory_analyze(normalize_in_place, "Kb", np.random.randint(1, 10, 10000).reshape(10, 100, 10))

Current memory usage is 0.58203125KB; Peak was 79.107421875KB


In [6]:
import numpy as np
from memory_profiler import memory_usage
interval_sec = 0.2
memory_changing = memory_usage((make_two_arrays), interval=interval_sec)
dict(zip(np.cumsum([interval_sec] * len(memory_changing)),
         memory_changing))

{0.2: 361.7265625,
 0.4: 361.7265625,
 0.6000000000000001: 401.3984375,
 0.8: 404.20703125,
 1.0: 404.20703125,
 1.2: 404.20703125,
 1.4: 404.20703125}

Давайте сравним расходы памяти в случае генератора и списка.

In [8]:
from memory_checking_1 import (stack_using_list, stack_using_generator)
n_images = 500
n_pixels = 1000
snapshot_interval = 0.2 # seconds
list_mem_usage = memory_usage((stack_using_list,(n_images, n_pixels)), interval=snapshot_interval)
gen_mem_usage = memory_usage((stack_using_generator,(n_images, n_pixels)), interval=snapshot_interval)

In [9]:
dict(zip(np.cumsum([snapshot_interval] * len(list_mem_usage)),
         list_mem_usage))

{0.2: 73.83984375,
 0.4: 74.3046875,
 0.6000000000000001: 174.11328125,
 0.8: 250.2734375,
 1.0: 320.66015625,
 1.2: 385.75390625,
 1.4: 443.7734375,
 1.5999999999999999: 263.0,
 1.7999999999999998: 184.2421875,
 1.9999999999999998: 275.1171875,
 2.1999999999999997: 427.5234375,
 2.4: 529.40625,
 2.6: 545.33203125,
 2.8000000000000003: 545.29296875,
 3.0000000000000004: 443.78125,
 3.2000000000000006: 169.92578125,
 3.400000000000001: 146.3671875,
 3.600000000000001: 215.49609375,
 3.800000000000001: 316.93359375,
 4.000000000000001: 373.53515625,
 4.200000000000001: 456.35546875,
 4.400000000000001: 553.87890625,
 4.600000000000001: 600.21484375,
 4.800000000000002: 555.828125,
 5.000000000000002: 415.22265625,
 5.200000000000002: 373.796875,
 5.400000000000002: 418.05859375,
 5.600000000000002: 542.16796875,
 5.8000000000000025: 366.64453125,
 6.000000000000003: 454.0234375,
 6.200000000000003: 496.68359375,
 6.400000000000003: 529.41015625,
 6.600000000000003: 587.28515625,
 6.80000

In [10]:
dict(zip(np.cumsum([snapshot_interval] * len(gen_mem_usage)),
         gen_mem_usage))

{0.2: 13.19140625,
 0.4: 15.19140625,
 0.6000000000000001: 35.83984375,
 0.8: 38.23828125,
 1.0: 38.23828125,
 1.2: 38.23828125,
 1.4: 38.23828125,
 1.5999999999999999: 38.23828125,
 1.7999999999999998: 38.23828125,
 1.9999999999999998: 38.23828125,
 2.1999999999999997: 38.23828125,
 2.4: 38.23828125,
 2.6: 38.23828125,
 2.8000000000000003: 38.23828125,
 3.0000000000000004: 38.23828125,
 3.2000000000000006: 38.23828125,
 3.400000000000001: 38.23828125,
 3.600000000000001: 38.23828125,
 3.800000000000001: 38.23828125,
 4.000000000000001: 30.62890625}

# Параллелизация: мультипроцессинг и многопоточность

Все, что мы делали до этого, мы делали на 1 ядре процессоре. Но как мы знаем, у каждого современного процессора больше 1 ядра, а у каждого ядра есть два потока!
Соответственно, мы можем выполнять идентичные друг другу процессы параллельно!
Однако, давайте сначала рассмотрим основные виды программ.

***CPU Bound*** означает, что скорость выполнения процесса ограничена скоростью центрального процессора. Задача, выполняющая вычисления на небольшом наборе чисел, например, перемножение небольших матриц, скорее всего, будет привязана к процессору.

***I/O Bound*** означает, что скорость выполнения процесса ограничена скоростью подсистемы ввода-вывода. Задача, которая обрабатывает данные с диска, например, подсчитывает количество строк в файле, скорее всего, будет связана с вводом-выводом.

***Memory bound*** означает, что скорость выполнения процесса ограничена объемом доступной оперативной памяти и скоростью доступа к ней. Задача, которая обрабатывает большие объемы данных в памяти, например, перемножение больших матриц, скорее всего, будет ограничена памятью.

***Cache bound*** означает, что скорость выполнения процесса ограничена объемом и скоростью доступного кэша. Задача, которая просто обрабатывает больше данных, чем помещается в кэш, будет ограничена кэшем.


Теперь рассмотрим способы ускорения программ.

***Параллелизм (parallelism)*** - одновременное выполнение нескольких операций.

***Поточность (threading)*** - это модель параллельного выполнения, при которой несколько потоков поочередно выполняют задачи. Один процесс может содержать несколько потоков. У Python сложные отношения с потоками благодаря его GIL, но эти подробности мы разбирать не будем. Главное запомнить, что поточность используется, когда нам нужно ускорить I/O задачи (задачи, связанные с импортом и выводом данных).
Нативный Python использует библиотеку **threading**

***Мультипроцессинг (multiprocessing)*** - это один из способов для осуществления параллелизма, которое предполагает распределение задач по центральным процессорам (CPU, или ядрам) компьютера. Многопроцессорная обработка хорошо подходит для задач, привязанных к процессору: в эту категорию обычно попадают жестко привязанные циклы и математические вычисления.
Нативный Python использует библиотеку **multiprocessing**

***Одновременность / многопоточность (concurrency)*** - это несколько более широкий термин, чем параллелизм. Он подразумевает, что несколько задач могут выполняться параллельно. (Существует поговорка, что одновременность не подразумевает параллельности).
Нативный Python использует библиотеку **concurrent.futures**

Обычную поточность мы тут рассматривать не будем. Давайте посмотрим, как реализуется одновременная многопоточность.

## Многопоточность

In [4]:
%%time
# Один поток
import requests
from parallelism.multithreading import urls
results = []
for url in urls:
    with requests.get(url) as src:
        results.append(src.content)

CPU times: user 256 ms, sys: 32.2 ms, total: 289 ms
Wall time: 2.38 s


In [46]:
%%time
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(4) as executor:  # 4 потока (2 ядра по 2 потока)
    results = executor.map(requests.get, urls)
results = list(map(lambda x: x.content, results))

CPU times: user 264 ms, sys: 52.2 ms, total: 316 ms
Wall time: 811 ms


## Мультипроцессинг

In [1]:
%%time
from multiprocessing import Pool
from parallelism.multiprocessing import if_prime

answer = 0

for i in range(1000000):
    answer += if_prime(i)

CPU times: user 5.84 s, sys: 79.3 ms, total: 5.91 s
Wall time: 6.14 s


In [2]:
%%time
with Pool(4) as p:  # 4 потока на 2 ядрах
    answer = sum(p.map(if_prime, list(range(1000000))))

CPU times: user 172 ms, sys: 77.6 ms, total: 250 ms
Wall time: 5.05 s


Давайте попробуем применить многопоточность на численных операциях и мультипроцессинг на операциях I/O.

In [50]:
%%time
with ThreadPoolExecutor(4) as executor:  # 4 потока (2 ядра по 2 потока)
    results = executor.map(if_prime, list(range(1000000)))

CPU times: user 30.4 s, sys: 4.21 s, total: 34.6 s
Wall time: 33.8 s


In [53]:
%%time
with Pool(4) as p:  # 4 потока на 2 ядрах
    answer = p.map(requests.get, urls)
answer = list(map(lambda x: x.content, answer))

CPU times: user 24.1 ms, sys: 65.5 ms, total: 89.6 ms
Wall time: 1.26 s


Как видим, многопоточность сильно хуже справилась с численными операциями, а мультипроцессинг - с операциями ввода-вывода.

## Joblib


In [3]:
%%time
from joblib import Parallel, delayed
results = sum(Parallel(n_jobs=-1, verbose=3)(delayed(if_prime)(i) for i in range(1000000)))  # n_jobs = -1 - все потоки

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 65544 tasks      | elapsed:    2.5s


CPU times: user 5.43 s, sys: 270 ms, total: 5.7 s
Wall time: 8.56 s


[Parallel(n_jobs=-1)]: Done 1000000 out of 1000000 | elapsed:    8.3s finished


## p_tqdm


In [4]:
%%time
from p_tqdm import p_map
results = p_map(if_prime, range(int(1e6)))

  0%|          | 0/1000000 [00:00<?, ?it/s]

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/iakubovskii/Репетитор/Development/venv/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/iakubovskii/Репетитор/Development/venv/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/iakubovskii/Репетитор/Development/venv/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/iakubovskii/Репетитор/Development/venv/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/iakubovskii/Репетитор/Development/venv/lib/python3.8/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwarg

KeyboardInterrupt: 

# Ассинхронность

https://realpython.com/async-io-python/
